# Assignment Final
### INFO-615

## Trucks Only Route

In [2]:
# Imported Packages
import numpy as np
import pandas as pd
import geopy.distance
import seaborn as sns
import geopandas as gpd
from geopandas import GeoSeries, GeoSeries
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2



## Travelling Salesman problem

### Importing Data

In [3]:
hpo_small= pd.read_csv("../../GIS Final Assignment/Part 1- Data Cleaning and Basic Assessment/hp_orange_small.csv")

#### For Small Farms Only 

In [4]:
# Adding a new row for the zero address( port address) 
new_row = pd.DataFrame({'Unname':0, 'Business Name':'Bronx Material Innovation Hub','Full Address':'1 Halleck St, Bronx, NY 10474', 
                        'Latitude': 40.8013201, 'Longitude': -73.8953683,'RegisteredAcre':0 , 'index_right':35, 'NAME': 'Bronx'},
                                                            index =[0])
    
# simply concatenate both dataframes
hpo_small = pd.concat([new_row, hpo_small]).reset_index(drop = True)
 



In [5]:

hpo_small = hpo_small.assign(ID=range(len(hpo_small)))

hpo_small

,Unname,Business Name,Full Address,Latitude,Longitude,RegisteredAcre,index_right,NAME,Unnamed: 0,RegisteredAcres,geometry,ID
0,0.0,Bronx Material Innovation Hub,"1 Halleck St, Bronx, NY 10474",40.801320,-73.895368,0.0,35,Bronx,NaN,NaN,NaN,0
1,NaN,Big M Direct LLC,PO Box 311 Pine Island NY 10969,41.297872,-74.459324,NaN,35,Orange,53.0,8.0,POINT (-74.459324 41.2978722),1
2,NaN,Chester Agricultural Center LLC,PO Box 269 Mountainville NY 10918,41.404068,-74.078050,NaN,35,Orange,112.0,4.0,POINT (-74.0780502 41.4040683),2
3,NaN,Chickidoo Hempstead,199 Tally Ho Road Middletown NY 10940,41.454206,-74.504500,NaN,35,Orange,113.0,0.5,POINT (-74.5045003 41.4542061),3
4,NaN,Denice Davis LLC,158 Pulaski Highway Pine Island NY 13408,41.302306,-74.449586,NaN,35,Orange,156.0,1.0,POINT (-74.4495859 41.3023059),4
5,NaN,Green Harvest Farms LLC,64A Upper Wisner Rd Warwick NY 10990,41.263815,-74.325382,NaN,35,Orange,270.0,2.0,POINT (-74.3253823 41.2638147),5
6,NaN,Gullybean Inc.,PO Box 269 Mountainville NY 10953,41.389303,-74.093897,NaN,35,Orange,284.0,10.0,POINT (-74.0938969 41.389303),6
7,NaN,GWM Distribution LLC,"320 Robinson Avenue, Suite 212 Newburgh NY 12550",41.513458,-74.019777,NaN,35,Orange,286.0,4.0,POINT (-74.0197768 41.5134577),7
8,NaN,Joe Sidoti,10 Hillcrest Drive Westtown NY 10998,41.320805,-74.563423,NaN,35,Orange,388.0,20.0,POINT (-74.5634231 41.3208053),8
9,NaN,Kanna Worx Industries LLC,49 Mountain View Terrace Walden NY 12586,41.545423,-74.126038,NaN,35,Orange,403.0,6.0,POINT (-74.1260376 41.5454228),9


In [6]:
coord_data= hpo_small[["Latitude", "Longitude"]].values.tolist()
ID= hpo_small["ID"].astype(int).values.tolist()


In [7]:
#### Creating a distance Matrix

In [8]:
from geopy import distance
import pandas as pd

def make_distance_matrix(coords, addresses):
    from_list = []
    to_list = []
    distance_list = []

    for a in list(range(len(coords))):
        for b in list(range(len(coords))):

            origin_coord = coords[a]
            dest_coord = coords[b]

            origin_name = addresses[a]
            dest_name = addresses[b]

            dist = distance.distance(origin_coord, dest_coord).miles

            from_list.append(origin_name)
            to_list.append(dest_name)
            distance_list.append(dist)

    # Create a matrix containing the results
    distance_matrix = pd.DataFrame()
    distance_matrix['from'] = from_list
    distance_matrix['to'] = to_list
    distance_matrix['distance'] = distance_list

    # Do a pivot table
    distance_matrix = distance_matrix.pivot_table(values='distance', index='from', columns='to')
    
    return distance_matrix

In [9]:
dm_s= make_distance_matrix(coord_data, ID)

#### To Ensure that the distance matrix has only integers and large number so it cannot be rounded off to zero

In [10]:

dm_s= dm_s*100

dm_s_f= dm_s.astype(int)

### Converting Distance Matrix to array 

In [11]:
distarray_s= dm_s_f.to_numpy()


In [12]:
distarray_s

array([[   0, 4518, 4267, 5513, 4509, 3903, 4187, 4956, 5002, 5273, 4492,
        4560, 4603, 4501, 3865, 4008, 5963, 4586, 4751, 4629, 4437, 5565],
       [4518,    0, 2113, 1104,   59,  735, 2002, 2725,  564, 2432,  160,
         170, 1436,  361,  659,  513, 2073,  157, 1735,  307,   84, 2358],
       [4267, 2113,    0, 2241, 2055, 1609,  131,  813, 2588, 1006, 2252,
        1974,  812, 1767, 1898, 1936, 1959, 2004,  690, 2421, 2065, 1328],
       [5513, 1104, 2241,    0, 1086, 1610, 2179, 2548,  970, 2061, 1211,
         998, 1430, 1015, 1683, 1553, 1185,  982, 1606, 1193, 1166, 1812],
       [4509,   59, 2055, 1086,    0,  698, 1944, 2666,  605, 2374,  216,
         118, 1377,  302,  644,  500, 2032,  113, 1676,  366,   80, 2303],
       [3903,  735, 1609, 1610,  698,    0, 1483, 2343, 1299, 2202,  804,
         703, 1164,  601,  290,  340, 2257,  735, 1462,  991,  654, 2255],
       [4187, 2002,  131, 2179, 1944, 1483,    0,  939, 2486, 1090, 2138,
        1866,  750, 1660, 1771, 

### Defining Data for the Travelling Saleperson Problem

In [13]:
def create_data_model(dma):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dma
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [14]:
data = create_data_model(distarray_s)
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [15]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  

In [16]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [17]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [18]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [19]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 16625 miles
Route for vehicle 0:
 0 -> 5 -> 14 -> 15 -> 13 -> 11 -> 17 -> 4 -> 20 -> 1 -> 10 -> 19 -> 8 -> 3 -> 16 -> 21 -> 9 -> 7 -> 18 -> 12 -> 2 -> 6 -> 0

